In [ ]:
import Pkg;
Pkg.activate(joinpath(@__DIR__, ".."));
Pkg.instantiate();
using DelimitedFiles
using CSV
using LinearAlgebra
using ForwardDiff
using RobotDynamics
using Ipopt
using MathOptInterface
const MOI = MathOptInterface
using Random;

In [ ]:
include("quadratic_cost.jl")
include("planar_quadruped.jl")
include("nlp.jl")
include("moi.jl")
include("costs.jl")
include("constraints.jl")
include("ref_traj.jl");

In [ ]:
# Dynamics model
model = PlanarQuadruped()
g, mb, lb, l1, l2 = model.g, model.mb, model.lb, model.l1, model.l2

# Some parameters
dt = 0.003
N = 61
times = range(0, dt * (N - 1), length=N)
k_trans = 21
n = 15
m = 5

# Initial condition. Currently, we assume the initial mode ID is 1
xinit = zeros(n)
xinit[1] = -lb / 2.5                # xb
xinit[2] = sqrt(l1^2 + l2^2) + 0.1  # yb
xinit[3] = -20 * pi / 180           # theta
xinit[6] = -lb                      # x2
xinit[7] = 0.2                      # y2
xinit[9] = -1.0                     # yb_dot
xinit[14] = -1.0                    # y2_dot
init_mode = 1

# Desired final state
xterm = zeros(n)
xterm[1] = -lb / 2           # xb
xterm[2] = sqrt(l1^2 + l2^2) # yb
xterm[6] = -lb;              # x2

# # Initial condition. Currently, we assume the initial mode ID is 2
# xinit = zeros(n)
# xinit[1] = lb / 2.5                 # xb
# xinit[2] = sqrt(l1^2 + l2^2) + 0.1  # yb
# xinit[3] = 20 * pi / 180            # theta
# xinit[4] = lb                       # x1
# xinit[5] = 0.2                      # y1
# xinit[9] = -3.0                     # yb_dot
# xinit[12] = -3.0                    # y1_dot
# init_mode = 2

# # Desired final state
# xterm = zeros(n)
# xterm[1] = lb / 2            # xb
# xterm[2] = sqrt(l1^2 + l2^2) # yb
# xterm[4] = lb;               # x1

In [ ]:
# Reference Trajectory
Xref, Uref = reference_trajectory(model, N, k_trans, xterm, init_mode, dt);

In [ ]:
# Objective
Q = Diagonal([10.0; 10.0; 1.0; 10.0; 10.0; 10.0; 10.0; 10.0; 10.0; 1.0; 10.0; 10.0; 10.0; 10.0; 0.0])
R = Diagonal(fill(1e-3, 5))
R[end, end] = 0.0
Qf = Q

obj = map(1:N-1) do k
    LQRCost(Q, R, Xref[k], Uref[k])
end
push!(obj, LQRCost(Qf, R * 0, Xref[N], Uref[1]));

In [ ]:
# Define the NLP
nlp = HybridNLP(model, obj, init_mode, k_trans, N, xinit, xterm);

In [ ]:
# Initial guess
Random.seed!(1)

# Uguess = [u + 0.1*randn(length(u)) for u in Uref]
# Xguess = [x + 0.1*randn(length(x)) for x in Xref]

# initialize Xguess
Xguess = [zeros(n) for x in Xref]
k_trans = nlp.k_trans

for k = 1:N
    if k <= k_trans
        Xguess[k] = xinit + (xterm - xinit) / (k_trans - 1) * (k - 1)
    else
        Xguess[k][1:14] = xterm[1:14]
    end
    
    Xguess[k][end] = dt * (k - 1)
end

# initialize Uguess
Uguess = [zeros(m) + rand(m) for u in Uref]
for k = 1: N-1
    Uguess[k][end] = dt
end

In [ ]:
Z0 = packZ(nlp, Xguess, Uref)

Z_sol, solver = solve(Z0, nlp, c_tol=1e-3, tol=1e-3)

In [ ]:
Z_sol[1:15] - xinit

In [ ]:
Z_sol[end-14:end] - xterm

In [ ]:
a = zeros(N-1)
for k = 1:N-1
    vb_x_k = Z_sol[8+20*(k-1)]
    vb_x_next = Z_sol[8+20*(k-1+1)]
    dt = Z_sol[20+20*(k-1)]
    a[k] = (vb_x_next - vb_x_k) / dt
    ay_dyn = contact1_dynamics(model, Z_sol[1+20*(k-1):14+20*(k-1)], Z_sol[16+20*(k-1):19+20*(k-1)])
    @show ay_dyn[9]
end

In [ ]:
dt = zeros(N-1)
for k = 1:N-1
    @show dt[k] = Z_sol[20 + 20 * (k-1)]
end

In [ ]:
# display(Z_sol)
writedlm("1.csv", Z_sol, ',')